In [128]:
year = '56'
pdf_filename = 'data/pdf-text' + year + '.txt'
major_filename = 'data/majors-adm' + year + '.csv'
output_filename = 'data/major-scores-adm' + year + '.csv'

In [129]:
lines = open(pdf_filename).readlines()
print(lines[:2])

['www.cuas.or.th ตารางแสดงสถตจานวนผสมคร จานวนรบ จานวนผผานฯ และคะแนนสงสด-ตาสด ในระบบ Admissions กลาง ประจาปการศกษา 2556 1.จฬาลงกรณมหาวทยาลย\n', 'รหส\n']


In [130]:
import re
major_number_pattern = r'^\d\d\d\d$'
major_number = re.compile(major_number_pattern)

In [131]:
old_key='xxxx'
old_items = []
pre_results = []
for l in lines:
    for item in l.split():        
        if major_number.match(item) and (old_items==[] or ('มหา' not in old_items[-1])):
            pre_results.append((old_key, old_items))
            old_items = []
            old_key = item
        else:
            old_items.append(item)
pre_results.append((old_key, old_items))
print(pre_results[:100])

[('xxxx', ['www.cuas.or.th', 'ตารางแสดงสถตจานวนผสมคร', 'จานวนรบ', 'จานวนผผานฯ', 'และคะแนนสงสด-ตาสด', 'ในระบบ', 'Admissions', 'กลาง', 'ประจาปการศกษา']), ('2556', ['1.จฬาลงกรณมหาวทยาลย', 'รหส', 'คณะ/สาขา', 'จานวน', 'คะแนน', 'สมคร', 'รบ', 'ผาน', 'สงสด', 'ตาสด', 'คาเฉลย', 'SD']), ('0001', ['คณะทนตแพทยศาสตร', '238', '30', '30', '25,714.7500', '23,569.9000', '24,007.1567', '430.0180']), ('0002', ['คณะเภสชศาสตร\uf70eสาขาวชาเภสชศาสตร', '313', '100', '100', '21,962.1500', '19,524.1000', '20,328.1035', '567.7893']), ('0003', ['คณะเภสชศาสตร\uf70eสาขาวชาการบรบาลทางเภสชกรรม', '391', '90', '90', '21,207.8000', '19,673.3000', '20,114.5861', '367.5579']), ('0004', ['คณะสตวแพทยศาสตร', '451', '150', '150', '23,225.6000', '19,924.2500', '20,525.8150', '490.5852']), ('0005', ['คณะสหเวชศาสตร\uf70eสาขาวชาเทคนคการแพทย', '481', '109', '109', '21,811.1500', '19,304.4500', '19,833.4908', '481.2220']), ('0006', ['คณะสหเวชศาสตร\uf70eสาขาวชากายภาพบาบด', '373', '82', '83', '20,852.4000', '18,871.8000', '19,315.3854

In [132]:
import csv
def read_major_title(filename):
    majors = []
    reader = csv.reader(open(filename))
    for row in reader:
        majors.append(row)
    return majors

majors = read_major_title(major_filename)
print(majors[:3])

[['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย']]


In [133]:
result_row = 0
results = []
for mdata in majors:
    number = mdata[0]
    org_result_row = result_row
    while (result_row < len(pre_results)) and (pre_results[result_row][0] != number):
        result_row += 1
    if result_row == len(pre_results):
        result_row = org_result_row
        continue
    results.append((mdata, pre_results[result_row]))
print(results[:3])

[(['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0001', ['คณะทนตแพทยศาสตร', '238', '30', '30', '25,714.7500', '23,569.9000', '24,007.1567', '430.0180'])), (['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0002', ['คณะเภสชศาสตร\uf70eสาขาวชาเภสชศาสตร', '313', '100', '100', '21,962.1500', '19,524.1000', '20,328.1035', '567.7893'])), (['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0003', ['คณะเภสชศาสตร\uf70eสาขาวชาการบรบาลทางเภสชกรรม', '391', '90', '90', '21,207.8000', '19,673.3000', '20,114.5861', '367.5579']))]


In [134]:
sd_pattern = r'[0-9,]+\.\d+'

def extract_at(data, loc):
    if re.match(r'\d+',data[loc - 6]):
        if loc != -1:
            return data[loc - 6:loc + 1]
        else:
            return data[loc - 6:]
    else:
        if loc != -1:
            return data[loc - 4:loc + 1]
        else:
            return data[loc - 6:]

def extract_zero_row(data):
    if data[-1] == '0':
        return data[-3:]
    try:
        i = -1
        while data[i] != '0':
            i -= 1
        return data[i-2:i+1]
    except:
        print('Error:',data)
        return data

def extract_scores(data):
    if re.match(sd_pattern,data[-1]):
        return extract_at(data, -1)
    else:
        i = -1
        try:
            while not re.match(sd_pattern,data[i]):
                i -= 1
            return extract_at(data, i)
        except:
            return extract_zero_row(data)
        
            
def extract_line(res):
    title = res[0]
    data = res[1][1]
    items = [title[0], title[1], title[3]] + extract_scores(data)
    return items

def print_line(res):
    print(",".join(extract_line(res)))

In [135]:
for r in results[105:115]:
    print_line(r)

0106,คณะวนศาสตร์ สาขาวิชาเทคโนโลยีเยื่อและกระดาษ,มหาวิทยาลัยเกษตรศาสตร์,56,25,25,16,773.4000,12,641.4000,14,101.7160,1,159.2280
0107,คณะวิทยาศาสตร์ สาขาวิชาคณิตศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,131,26,26,16,905.3000,15,094.7500,15,788.4038,469.1477
0108,คณะวิทยาศาสตร์ สาขาวิชาเคมี,มหาวิทยาลัยเกษตรศาสตร์,216,35,35,18,038.6500,15,908.2000,16,333.8629,465.6570
0109,คณะวิทยาศาสตร์ สาขาวิชาเคมีอุตสาหกรรม,มหาวิทยาลัยเกษตรศาสตร์,136,20,20,17,534.6500,15,705.7000,16,240.4300,477.2038
0110,คณะวิทยาศาสตร์ สาขาวิชาชีววิทยา,มหาวิทยาลัยเกษตรศาสตร์,620,40,40,16,920.1000,15,436.7500,15,860.3900,406.0621
0111,คณะวิทยาศาสตร์ สาขาวิชาสัตววิทยา,มหาวิทยาลัยเกษตรศาสตร์,295,40,40,16,688.8500,14,494.6500,15,097.4475,567.8069
0112,คณะวิทยาศาสตร์ สาขาวิชาพฤกษศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,132,30,30,15,979.3500,13,642.6500,14,383.3750,658.7578
0113,คณะวิทยาศาสตร์ สาขาวิชาพันธุศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,111,55,52,16,540.2500,11,046.5500,14,475.1702,1,253.2080
0114,คณะวิทยาศาสตร์ สาขาวิชาจุลชีววิทยา,มหาวิทยาลัยเกษต

In [136]:
def save_result(results, filename):
    header = ['รหัส','คณะ/สาขา','มหาวิทยาลัย','สมัคร','รับ','ผ่าน','สูงสุด','ต่ำสุด','เฉลี่ย','SD']
    writer = csv.writer(open(filename,'w'))
    writer.writerow(header)
    for r in results:
        writer.writerow(extract_line(r))

In [137]:
save_result(results,output_filename)